In [1]:
import numpy as np
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
/home/martin/anaconda3/envs/pi2/lib/python3.7/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
params = utils.yaml_to_dict(os.path.join('..','config.yml'))
params['data_dir'] = os.path.join('..',params['data_dir'])
params['model_dir'] = os.path.join('..',params['model_dir'])

## Primer paso: preprocesar los datos 
Hacerlo solo una vez

In [3]:
preprocessing.download_data(params)
preprocessing.extract_data(params)
preprocessing.generate_spectogram_images(params)
preprocessing.make_id_label_map(params)
preprocessing.split_data(params)

Window size is bigger than record segment: specgram_matrix_S-avanza9_segment22.png
Window size is bigger than record segment: specgram_matrix_S-izquierda7_segment6.png
Window size is bigger than record segment: specgram_matrix_S-izquierda9_segment25.png
Window size is bigger than record segment: specgram_matrix_S-frena10_segment24.png
Window size is bigger than record segment: specgram_matrix_S-frena5_segment28.png
Window size is bigger than record segment: specgram_matrix_S-acelera11_segment24.png
Window size is bigger than record segment: specgram_matrix_S-acelera8_segment0.png
Window size is bigger than record segment: specgram_matrix_S-acelera8_segment1.png
Window size is bigger than record segment: specgram_matrix_S-acelera7_segment16.png
Window size is bigger than record segment: specgram_matrix_S-acelera6_segment15.png
Window size is bigger than record segment: specgram_matrix_S-acelera5_segment11.png
Window size is bigger than record segment: specgram_matrix_S-acelera4_segment0

## Segund Paso: instanciar el modelo

In [4]:
inputs = tf.keras.layers.Input(shape=(110,480, 3))
net = model.ModelArchitecture(num_classes=params['num_classes'])
x = net(inputs, training=False)
net.summary()

W0727 10:28:17.901544 140469068961600 deprecation.py:506] From /home/martin/anaconda3/envs/pi2/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model_architecture"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 240, 3)        444       
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 240, 3)        12        
_________________________________________________________________
activation (Activation)      (None, 55, 240, 3)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 48, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1584)              0         
_________________________________________________________________
dense (Dense)                (None, 12)                19020     
_________________________________________________________________
batch_normalization_1 (Batch (None, 12)         

## Tercer paso: crear el generador

In [5]:
print('Data in train')
train_generator = data.make_datagenerator(params, mode='training')
print('Data in validation')
val_generator = data.make_datagenerator(params,mode='validation')

Data in train
Found 2959 images.
Data in validation
Found 30 images.


## Cuarto paso: correr el modelo

In [6]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(params['model_dir'], 'tf_ckpt'), 
    save_weights_only=True, 
    verbose=1,
    period=5)

tb_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(params['model_dir'], 'logs'))

optimizer = tf.keras.optimizers.Adam(lr=0.001)

steps_per_epoch = train_generator.n // params['batch_size']
validation_steps = val_generator.n // params['batch_size']

net.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
net.fit_generator(
    train_generator, 
    steps_per_epoch=steps_per_epoch, 
    epochs=params['num_epochs'],
    workers=4,
    validation_data=val_generator, 
    validation_steps=validation_steps,
    callbacks=[cp_callback,tb_callback])

W0727 10:28:18.400288 140469068961600 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Epoch 1/25
295/295 [==============================] - 21s 72ms/step - loss: 2.7561 - sparse_categorical_accuracy: 0.1387 - val_loss: 2.1842 - val_sparse_categorical_accuracy: 0.2333
Epoch 2/25
295/295 [==============================] - 24s 82ms/step - loss: 1.8185 - sparse_categorical_accuracy: 0.3320 - val_loss: 1.4793 - val_sparse_categorical_accuracy: 0.5667
Epoch 3/25
295/295 [==============================] - 23s 77ms/step - loss: 1.2722 - sparse_categorical_accuracy: 0.6053 - val_loss: 1.0935 - val_sparse_categorical_accuracy: 0.5333
Epoch 4/25
295/295 [==============================] - 22s 73ms/step - loss: 0.9757 - sparse_categorical_accuracy: 0.7860 - val_loss: 0.8452 - val_sparse_categorical_accuracy: 0.7667
Epoch 5/25
294/295 [============================>.] - ETA: 0s - loss: 0.8047 - sparse_categorical_accuracy: 0.8408
Epoch 00005: saving model to ../checkpoints/tf_ckpt
295/295 [==============================] - 22s 75ms/step - loss: 0.8054 - sparse_categorical_accuracy: 0.

In [7]:
#fpr, tpr, thresholds = roc_curve(y, probs)

In [8]:
# Quintero paso: evaluar el modelo

In [9]:

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

Y_pred = net.predict_generator(val_generator, validation_steps)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(val_generator.data, y_pred))
print('Classification Report')
target_names = ['Cats', 'Dogs', 'Horse']
print(classification_report(val_generator.data, y_pred, target_names=["arranca", "acelera", "pita", "izquierda",
                                                                      "frena", "detente", "retrocede", "gira", "avanza", "derecha"]))

Confusion Matrix
[[0 0 1 0 0 0 0 1 1 0]
 [1 0 0 0 1 1 0 0 0 1]
 [1 0 1 0 0 0 1 0 0 0]
 [0 0 1 0 1 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 1 1]
 [0 0 0 0 1 0 1 0 0 1]
 [1 1 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0 0 0 2]]
Classification Report
              precision    recall  f1-score   support

     arranca       0.00      0.00      0.00         3
     acelera       0.00      0.00      0.00         4
        pita       0.25      0.33      0.29         3
   izquierda       0.00      0.00      0.00         2
       frena       0.00      0.00      0.00         4
     detente       0.00      0.00      0.00         3
   retrocede       0.00      0.00      0.00         3
        gira       0.00      0.00      0.00         2
      avanza       0.00      0.00      0.00         2
     derecha       0.33      0.50      0.40         4

   micro avg       0.10      0.10      0.10        30
   macro avg       0.06      0.08      0.07        30
weighted avg       0.07   